In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import gmaps
# Google developer API key
from config import gkey
# Access maps with unique API key
gmaps.configure(api_key=gkey)

%matplotlib inline
sns.set_style("darkgrid")

ModuleNotFoundError: No module named 'config'

In [4]:
# country codes conversion
country_csv = ("country_codes.csv")
country_df = pd.read_csv(country_csv)
country_df2 = country_df[["Alpha-2 code","Alpha-3 code"]].copy()
country_df3 = country_df2.rename(columns={'Alpha-3 code': 'NOC'}, inplace = False)
country_df3.head()

,Alpha-2 code,NOC
0,AF,AFG
1,AX,ALA
2,AL,ALB
3,DZ,DZA
4,AS,ASM


In [ ]:
path = ("athlete_events.csv")
df_events = pd.read_csv(path)


#df_events["join_id"] = df_events["NOC"] + df_events["Year"].astype(str)
# summer_df = df_events[df_events['Season'] == 'Summer']
# summer_df["BMI"] = summer_df["Weight"]/((summer_df["Height"]/100)**2)
# Add numerical value for medals? 3 = gold, 2= silver, 1 bronze and no medal = 0
df_events_a = pd.merge(df_events, country_df3, on="NOC")
df_events_a['join_id'] = df_events_a["NOC"] + df_events_a["Year"].astype(str)
df_events_a.head(10)



In [ ]:
path3 = ("country_lat_long.csv")
lat_long_df = pd.read_csv(path3)
lat_long_df2 = lat_long_df.rename(columns={'country': 'Alpha-2 code'})
lat_long_df2


In [ ]:
# lat_long_df2['join_id'] =  lat_long_df2["Alpha-2 code"] + lat_long_df2["Year"].astype(str)
df_events_b = pd.merge(df_events_a, lat_long_df2, on="Alpha-2 code")
df_events_b.head()

In [ ]:
# population
path4 = ("population_by_country.csv")
pop_df = pd.read_csv(path4)
pop_df['join_id'] =  pop_df["NOC"] + pop_df["Year"].astype(str)
pop_df

In [ ]:
df_events_c = pd.merge(df_events_b, pop_df, on="join_id")
df_events_c


In [ ]:
df_country_summ = df_events_b.groupby(["NOC","Season"]).size().reset_index(name="Count")
df_country_summ

In [ ]:
df_country_sum2 = df_country_summ.pivot(index='NOC', columns='Season', values=['Count'])
df_country_sum2.replace(np.nan,0,inplace=True)
df_country_sum2.reset_index(inplace=True)
df_country_sum2.columns = [' '.join(col).strip() for col in df_country_sum2.columns.values]

df_country_sum2.head()

In [ ]:
conditions = [
    (df_country_sum2['Count Summer']> 0) & (df_country_sum2['Count Winter'] > 0),
    (df_country_sum2['Count Summer']> 0) & (df_country_sum2['Count Winter'] == 0),
    (df_country_sum2['Count Summer'] == 0) & (df_country_sum2['Count Winter'] > 0)]
choices = [2, 1, 3]

df_country_sum2['summer_winter'] = np.select(conditions, choices, default=0)
df_country_sum2.columns

In [ ]:
df_events_d = pd.merge(df_events_b, df_country_sum2, on="NOC")
df_events_d

In [ ]:
# subset for northern hemisphere
both_north = df_events_d[df_events_d["latitude"] >= 0]

# Use the 'hue' argument to provide a factor variable
sns.lmplot( x="longitude", y="latitude", data=both_north, fit_reg=False, hue='summer_winter', legend=False)
plt.legend(loc='upper left')
plt.show()

In [ ]:
# subset for northern hemisphere
both_south = df_events_d[df_events_d["latitude"] < 0]

# Use the 'hue' argument to provide a factor variable
sns.lmplot( x="longitude", y="latitude", data=both_south, fit_reg=False, hue='summer_winter', legend=False)
plt.legend(loc='upper left')
plt.show()

In [ ]:
# subset for northern hemisphere
summer_only = df_events_d[df_events_d["summer_winter"] == 1]
summer_only2 = summer_only[['latitude','longitude']]

# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

summer_only2


In [ ]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(summer_only2)
# Add the layer to the map
fig.add_layer(markers)
fig